<a href="https://colab.research.google.com/github/anilbhatt1/Deep_Learning_EVA4_Phase1/blob/master/EVA4_P1_S4_0_9929_No_FC_Regularization_Batch_Norm_GAP_1x1_Dropout_Pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1,bias=False) # RF -> 3, ip - 28x28x1  op - 28x28x16
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1,bias=False) # RF -> 5, ip - 28x28x16 o/p - 28x28x16
        self.conv1a = nn.Conv2d(32,16,1,bias=False) 
        self.bn1   = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2) #  RF -> 6, ip - 28x28x16 o/p - 14x14x16
        self.drop1 = nn.Dropout2d(0.08)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1,bias=False) #  RF -> 10, ip - 14x14x16 o/p - 14x14x16
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1,bias=False) # RF -> 14, ip 14x14x16 o/p - 14x14x16
        self.conv1b = nn.Conv2d(32,16,1,bias=False)
        self.bn2   = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2) #  RF -> 16, ip - 14x14x16 o/p - 7x7x16
        self.drop2 = nn.Dropout2d(0.08)
        self.conv5 = nn.Conv2d(16, 16, 3, bias=False) # RF -> 24, ip 7x7x16 o/p - 5x5x16
        self.conv1c = nn.Conv2d(16,10,1,bias=False)
        self.gap   = nn.AdaptiveAvgPool2d((1,1))

    def forward(self, x):
        x = F.relu(self.conv2(F.relu(self.conv1(x))))
        x = self.drop1(self.pool1(self.bn1(self.conv1a(x))))
        x = F.relu(self.conv4(F.relu(self.conv3(x))))
        x = self.drop2(self.pool2(self.bn2(self.conv1b(x))))
        x = self.conv1c(F.relu(self.conv5(x)))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             288
            Conv2d-2           [-1, 32, 28, 28]           9,216
            Conv2d-3           [-1, 16, 28, 28]             512
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
         Dropout2d-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           2,304
            Conv2d-8           [-1, 32, 14, 14]           4,608
            Conv2d-9           [-1, 16, 14, 14]             512
      BatchNorm2d-10           [-1, 16, 14, 14]              32
        MaxPool2d-11             [-1, 16, 7, 7]               0
        Dropout2d-12             [-1, 16, 7, 7]               0
           Conv2d-13             [-1, 16, 5, 5]           2,304
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)

for epoch in range(1, 21):
    print('Epoch:',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.1048, Accuracy: 9672/10000 (96.72%)

Epoch: 2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0216, Accuracy: 9925/10000 (99.25%)

Epoch: 19


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0271, Accuracy: 9911/10000 (99.11%)

Epoch: 20


loss=0.0844767689704895 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 57.22it/s]



Test set: Average loss: 0.0217, Accuracy: 9924/10000 (99.24%)

